In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

In [2]:
# Read all data files of one category.
import os

trajectories_per_category = {}
categories = ['alive', 'fibro', 'plastic']
downsample_steps = [2, 5, 5]
for category, downsample_step in zip(categories, downsample_steps):
    data_dir = 'data/JulianTrajs/' + category
    trajectories_per_category[category] = []  # dimensions: trajectory index -> time step -> coordinate (x, y, z)
    filenames = os.listdir(data_dir)    
    for filename in filenames:
        trajectory = np.genfromtxt(os.path.join(data_dir, filename))
        for start in range(downsample_step):
            end = -(downsample_step - start)
            sliced_trajectory = trajectory[start:end:downsample_step]
            trajectories_per_category[category].append(sliced_trajectory)
    trajectories_per_category[category] = np.array(trajectories_per_category[category])
    print "Category {}: Found {} files, created {} trajectories".format(category, len(filenames), len(trajectories_per_category[category]))

Category alive: Found 65 files, created 130 trajectories
Category fibro: Found 69 files, created 345 trajectories
Category plastic: Found 177 files, created 885 trajectories


In [3]:
def to_velocities(x, multiple_trajectories=False):
    return np.diff(x, axis=int(multiple_trajectories)) / 5.
# TODO: Does only work for single trajectory right now, extend to multiple trajectories if needed.
def to_positions(x):
    return np.append(np.zeros((1, 3)), np.cumsum(x * 5., axis=0), axis=0)

In [4]:
for category in trajectories_per_category:
    trajectories_per_category[category] = to_velocities(trajectories_per_category[category], multiple_trajectories=True)

In [5]:
# Normalize trajectories to [-1, 1] for the LSTM network (it outputs values between -1 and 1 by default)
# TODO: Do not use min/max values here but fixed value to generalize to other datasets.
min_value = np.min([np.min(traj) for traj in trajectories_per_category.values()])
max_value = np.max([np.max(traj) for traj in trajectories_per_category.values()])
def normalize(x):
    return np.interp(x, [min_value, max_value], [-1., 1.])
def denormalize(x):
    return np.interp(x, [-1., 1.], [min_value, max_value])

In [6]:
for category in trajectories_per_category:
    trajectories_per_category[category] = normalize(trajectories_per_category[category])

In [7]:
# TODO: Split into training/test data here.
# TODO: Possible to use validation data in this scenario?


# Split trajectories into short parts following. Save the next position for each part.
# See also https://github.com/fchollet/keras/blob/master/examples/lstm_text_generation.py

category = 'fibro'
maxlen = 20  # Change this to see how strong the current position depends on the positions long ago. REPHRASE THIS
step = 3  # Change this to vary redundancy. 
mini_trajectories = []
next_values = []

for trajectory in trajectories_per_category[category]:
    for i in range(0, len(trajectory) - maxlen, step):
        mini_trajectories.append(trajectory[i:i+maxlen])
        next_values.append(trajectory[i+maxlen] - trajectory[i])
        
mini_trajectories = np.array(mini_trajectories)
next_values = np.array(next_values)

print 'Created', len(mini_trajectories), 'data samples for category', category

Created 20700 data samples for category fibro


In [8]:
# Set up the network.
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM

model = Sequential()

# TODO: Try stateful for the same trajectory
# TODO: Play around with output_dim here. 
model.add(LSTM(input_shape=(maxlen, 3), output_dim=100, return_sequences=True))
#model.add(Dropout(0.3))
#model.add(LSTM(output_dim=10, return_sequences=True))
model.add(LSTM(output_dim=100, return_sequences=False))
model.add(Dense(output_dim=3))
model.add(Activation('linear'))

model.compile(loss='mean_squared_error', optimizer='rmsprop')

Using Theano backend.
D:\Python\27_32bit\lib\site-packages\theano\tensor\signal\downsample.py:5: UserWarning: downsample module has been moved to the pool module.
  warnings.warn("downsample module has been moved to the pool module.")


In [12]:
# Train the model.
hist = model.fit(mini_trajectories, next_values, nb_epoch=10, batch_size=50, verbose=1)

# TODO: See if GPU is better for RNN (according to keras docs)
# TODO: Play with batch_size --> esp, does it influence final accuracy or only learning time?
# TODO: Other implementation: learn on single trajectory, X = trajectory[:-1], y = trajectory[1:], then repeat this for the other trajectories
#       Input and output dimension 3 in this case; does the network have to be stateful?

# TODO: How is the accuracy calculated here?

Epoch 1/10
20700/20700 [==============================] - 127s - loss: 0.0030   
Epoch 2/10
20700/20700 [==============================] - 108s - loss: 0.0028   
Epoch 3/10
20700/20700 [==============================] - 87s - loss: 0.0027    
Epoch 4/10
20700/20700 [==============================] - 80s - loss: 0.0026    
Epoch 5/10
20700/20700 [==============================] - 96s - loss: 0.0025    
Epoch 6/10
20700/20700 [==============================] - 80s - loss: 0.0024    
Epoch 7/10
20700/20700 [==============================] - 79s - loss: 0.0023    
Epoch 8/10
20700/20700 [==============================] - 87s - loss: 0.0023    
Epoch 9/10
20700/20700 [==============================] - 97s - loss: 0.0022    
Epoch 10/10
20700/20700 [==============================] - 105s - loss: 0.0022   


In [ ]:
plt.plot(range(1, len(hist.history['loss']) + 1), hist.history['loss'])

In [ ]:
# Test the model predictions on the training data. 
predicted = denormalize(model.predict(mini_trajectories))
expected = denormalize(next_values)

In [ ]:
zip(predicted[100:120], expected[100:120])

In [ ]:
# Rough measure for deviation.
np.sum(np.abs(predicted - expected)) / len(predicted) / 3.
# TODO: Compare with constant or random predictor

In [ ]:
# TODO: Other possible idea: Make matrix around current position with probabilities as output
# TODO: CLassification: feed in points step by step, see how classification probability changes
# TODO: Make the network predict persistence etc

In [ ]:
# Constant predictor (take last position of trajectory as prediction for next position).
constant_predicted = denormalize(mini_trajectories[:, -1])
np.sum(np.abs(constant_predicted - expected)) / len(predicted) / 3.

In [ ]:
%matplotlib tk
# Predict new trajectory by using one mini trajectory as seed.
num_predictions = 237

seed = mini_trajectories[np.random.randint(len(mini_trajectories))]
generated_trajectory = list(seed)

for i in range(num_predictions):
    next_value = model.predict(seed[np.newaxis])[0]  # TODO: Maybe introduce some randomness/temperature.
    #next_value += (np.random.rand(3) - 0.5) 
        
    #print next_value[0]
    generated_trajectory.append(next_value)
    seed = np.append(seed, next_value[np.newaxis], axis=0)[1:]
    
generated_trajectory = np.array(generated_trajectory)
generated_trajectory = denormalize(generated_trajectory)
generated_trajectory = to_positions(generated_trajectory)
#plt.plot(generated_trajectory[:, 0], generated_trajectory[:, 1])
plt.plot(generated_trajectory[:, 0][:50], generated_trajectory[:, 1][:50])
plt.plot(generated_trajectory[:, 0][49:], generated_trajectory[:, 1][49:])

lim = 1000

plt.xlim(-lim, lim)
plt.ylim(-lim, lim)

plt.gca().set(adjustable='box-forced', aspect='equal')
plt.show()

In [ ]:
%matplotlib tk
for i in range(20):    
    num_predictions = 237

    seed = mini_trajectories[np.random.randint(len(mini_trajectories))]
    generated_trajectory = list(seed)

    for i in range(num_predictions):
        next_value = model.predict(seed[np.newaxis])[0]  # TODO: Maybe introduce some randomness/temperature.
        #next_value += (np.random.rand(3) - 0.5) 
        #print next_value[0]
        generated_trajectory.append(next_value)
        seed = np.append(seed, next_value[np.newaxis], axis=0)[1:]

    generated_trajectory = np.array(generated_trajectory)
    generated_trajectory = denormalize(generated_trajectory)
    generated_trajectory = to_positions(generated_trajectory)
    plt.plot(generated_trajectory[:, 0], generated_trajectory[:, 1])
    
# lim = 700
# plt.xlim(-lim, lim)
# plt.ylim(-lim, lim)

plt.gca().set(adjustable='box-forced', aspect='equal')
plt.show()

In [ ]:
for mt in mini_trajectories:
    plt.plot(mt[:, 0], mt[:, 1], 'o')
plt.axvline(0)
plt.axhline(0)

In [ ]:
mini_trajectories[:, :, 0].shape

In [ ]:
rt = np.cumsum(mini_trajectories[0:1], axis=1)
for r in rt:
    plt.plot(r[:, 0], r[:, 1])

In [ ]:
rt